## Web crawler


import modules


In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

from fake_useragent import UserAgent




import time
import numpy as np
import pandas as pd
import pickle
import csv
import redis
import requests

initialize args


In [7]:
service = Service()
options = webdriver.ChromeOptions()
options.page_load_strategy = 'eager'
options.add_argument("--enable-javascript")

ua = UserAgent()
user_agent = ua.random
options.add_argument(f'--user-agent={user_agent}')
# options.add_argument('--headless')  # 啟動Headless 無頭



extract publisher, creator, year from raw data


In [3]:
def process_rawData(creatorRaw, publisherRaw):
       creator = creatorRaw.removesuffix(', 著')
       split = publisherRaw.split(',')
       year = split[-1]
       year = year.split('[')[0]
       publisher = split[0].split(':')[-1].replace('\n', ' ')
       publisher = publisher.removeprefix(' ')
       
       return creator, publisher, year
       

crawler function


In [4]:
def crawl_book(bookTitle):
       driver = webdriver.Chrome(service=service, options=options)
       wait = WebDriverWait(driver, timeout=10, poll_frequency=1)
       baseUrl = 'https://metadata.ncl.edu.tw/blstkmc/blstkm#tudorkmtop'
       driver.get(baseUrl)
       
       # set search configs
       x1 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[2]/td[1]/select'
       select = Select(wait.until(
                     EC.presence_of_element_located((By.XPATH, x1))
                     ))
       select.select_by_value('n_TI')
       x2 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/select'
       select = Select(driver.find_element(By.XPATH, x2))
       select.select_by_value('C')
       # enter book name
       x3 = '//*[@id="hotname"]'
       query = driver.find_element(By.XPATH, x3)
       query.clear()
       query.send_keys(bookTitle)
       # press search
       x4 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[5]/td/input[1]'
       searchBtn = wait.until(
                     EC.presence_of_element_located((By.XPATH, x4))
                     )
       searchBtn.click()
       
       
       try:
              # sort by year , ascending
              x6 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[1]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/select'
              select = Select( wait.until(
                     EC.presence_of_element_located((By.XPATH, x6))
              ))
              select.select_by_value('YR')
              x7 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[1]/table/tbody/tr/td[3]/table/tbody/tr/td[3]/select'
              select = Select( wait.until(
                     EC.presence_of_element_located((By.XPATH, x7))
              ))
              select.select_by_value('0')
              x8 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[1]/table/tbody/tr/td[2]/input'
              sortBtn =  wait.until(
                     EC.presence_of_element_located((By.XPATH, x8))
              )
              sortBtn.click()
              
              xCreator = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[5]/td/table/tbody/tr[2]/td[3]'
              xPublisher  = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[5]/td/table/tbody/tr[2]/td[4]'
              creatorRaw =   wait.until(
                     EC.presence_of_element_located((By.XPATH, xCreator))
              ).text
              publisherRaw =  wait.until(
                     EC.presence_of_element_located((By.XPATH, xPublisher))
              ).text
              creator, publisher, year = process_rawData(creatorRaw, publisherRaw)
       except:
              creator, publisher, year = 'error', 'error', 'error'
              print('error', bookTitle)  
       driver.close()  
       
       return pd.Series( [bookTitle, creator, publisher, year], index=['title','creator','publisher','year'])


test one book 

In [8]:
crawl_book('三體')

title           三體
creator        劉慈欣
publisher      貓頭鷹
year          2023
dtype: object

In [26]:
df = pd.read_csv('書籍.csv')
df = df.dropna().reset_index()
print(df['書名'])

0               龍鱗焰火
1               龍族秘錄
2              黑暗的左手
3               黃海童話
4        非理性時代 牛頓加農砲
           ...      
496        [簡]納米魔幻兵團
497    [簡]現代中國科幻文學主潮
498     黃禍 3/3黃禍反撲世界
499     黃禍 1/3大陸內戰爆發
500    黃禍  2/3台灣生死存亡
Name: 書名, Length: 501, dtype: object


In [7]:
errorIndex = []
start = 400
checkpoint = 50
new_df = pd.DataFrame()

In [8]:
for i in range(start, start+checkpoint):
      series = crawl_book(df['書名'][i])
      if(series['creator']== 'error'):
            errorIndex.append(i) 
      new_df = pd.concat([new_df, series],axis=1)
      if(i%10 == 0):
            if(i!=0): 
                  print()
            print(f'done{i}/{start+checkpoint}', end = '')      
      print('.', end = '')

new_df.to_csv(f'./csv/{start}-{start+checkpoint}.csv')
start+=checkpoint


error Star Trek the Next Generation: Perchance to Dream

done400/450.error Star Trek the Next Generation: Metamorphosis
.error Star Trek the Next Generation: Doomsday World
.error Star Trek the Next Generation: Contamination
.error Star Trek the Next Generation: Chains of Command
.error Star Trek the Next Generation Immortal Coil
.error Star Trek the Next Generation Gateway
.error Star Trek S.C.E. Book Two
.error Star Trek S.C.E. Book One
.error STAR TREK Roleplaying Game Player`s Guide
.error Star Trek Roleplaying Game Narrator's Guide

done410/450.error Star Trek ds9
.error Star Trek Deep Space 9: Avatar Book Two
.error Star Trek Deep Space 9: Avatar Book One
.error Star Trek Deep Space 9 Section 31
.error Star Trek Cookbook
.....
done420/450....error Science Fiction SF: The Illustrated Encyclopedia
.error Science Fiction of the 20th Century
.error Robots and Aliens
.error Robot City Book 5
.error Robot City Book 4 Prodigy
.error Robot City Book 2 Supicion
.
done430/450..error Quotab

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome-headless-shell=127.0.6533.120)
Stacktrace:
0   chromedriver                        0x0000000104d85024 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x0000000104d7d700 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x000000010498c694 cxxbridge1$string$len + 88116
3   chromedriver                        0x00000001049d1860 cxxbridge1$string$len + 371200
4   chromedriver                        0x00000001049c73a8 cxxbridge1$string$len + 329032
5   chromedriver                        0x00000001049c6e38 cxxbridge1$string$len + 327640
6   chromedriver                        0x0000000104a0848c cxxbridge1$string$len + 595500
7   chromedriver                        0x00000001049c5474 cxxbridge1$string$len + 321044
8   chromedriver                        0x00000001049c60e4 cxxbridge1$string$len + 324228
9   chromedriver                        0x0000000104d4ca08 cx

In [34]:
errorIndex=[]
new_df=pd.DataFrame()

save var

In [32]:
with open(f'./var/var{start-checkpoint}-{start}.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([start, checkpoint, errorIndex, new_df], f)

load var

In [33]:
with open(f'./var/var{start-checkpoint}-{start}.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    start,checkpoint,errorIndex,new_df_ = pickle.load(f)

### combine data

In [97]:
df_ = pd.DataFrame()
for i in range(7):
       s = i*50
       e = s+50
       df_ =  pd.concat([df_,pd.read_csv(f'./csv/{s}-{e}.csv').transpose()[1:]])
print(df_)

              
       

                         0      1                   2          3
0                     龍鱗焰火   希爾 著                奇幻基地       2018
0.1                   龍族秘錄    蘇逸平              中國社會科學      2001.
0.2                  黑暗的左手  勒瑰恩 著                木馬文化       2020
0.3                   黃海童話    黃炳煌                  九歌   20061010
0.4            非理性時代 牛頓加農砲  堪易士 著  奇幻基地出版 家庭傳媒城邦分公司發行     民97.04
...                    ...    ...                 ...        ...
0.45          TNG: KAHLESS  error               error      error
0.46           TNG: IMZADI  error               error      error
0.47  TNG: FORTUNE'S LIGHT  error               error      error
0.48     TNG: DESCENT (兩本)  error               error      error
0.49      TNG: DARK MIRROR  error               error      error

[437 rows x 4 columns]


In [127]:
row, column = df_.shape

print(df_.values[0][1])
for i in range(row):
      
       df.loc[df['書名']==df_.values[i][0],'creator'] = df_.values[i][1]
       df.loc[df['書名']==df_.values[i][0],'publisher'] = df_.values[i][2]
       df.loc[df['書名']==df_.values[i][0],'year'] = df_.values[i][3]
print(df)
df.to_excel('out.xlsx')

希爾 著
     index             書名               ISBN          位置  \
0        0           龍鱗焰火        9.78987E+12          未知   
1        1           龍族秘錄      957-569-247-0  公關－會長－美工用具   
2        2          黑暗的左手      986-7399-03-X        艾波小姐   
3        3           黃海童話      957-444-348-5  公關－會長－美工用具   
4        4    非理性時代 牛頓加農砲  978-986-6712-14-2         大書櫃   
..     ...            ...                ...         ...   
496    525      [簡]納米魔幻兵團  978-7-5379-7075-4        艾波小姐   
497    526  [簡]現代中國科幻文學主潮  978-7-5366-9443-9        艾波小姐   
498    527   黃禍 3/3黃禍反撲世界      957-645-715-7        艾波小姐   
499    528   黃禍 1/3大陸內戰爆發      957-645-713-0        艾波小姐   
500    529  黃禍  2/3台灣生死存亡      957-645-714-9        艾波小姐   

                                                   url creator  \
0          https://cdn.cite.com.tw/images/1/1HB109.jpg    希爾 著   
1    https://upload.wikimedia.org/wikipedia/zh/9/90...     蘇逸平   
2    https://img1.doubanio.com/view/subject/m/publi...   勒瑰恩 著   
3    https

In [6]:


# 連接到 Redis 資料庫
r = redis.Redis(
  host='helpful-mako-37953.upstash.io',
  port=6379,
  password='AZRBAAIncDEzNDZlNjU0YTViNDE0YmE4YmQ4YWJkZjE1MjU3ZmU2NnAxMzc5NTM',
  ssl=True
)

# 定義要上傳的 hash 資料
hash_name = "book:9789863840053"
hash_data = {
    "name": "極限  上",
    "cover": "https://cdn.cite.com.tw/images/a/A1010261.jpg",
    "id": "9789863840053"
}

# 上傳 hash 資料到 Redis
r.hmset(hash_name, hash_data)

# 驗證資料是否上傳成功
retrieved_data = r.hgetall(hash_name)
print(retrieved_data)

/var/folders/jd/741sx0vn7h98nzy_4kpsbgg40000gn/T/ipykernel_83950/3375802425.py:18: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset(hash_name, hash_data)


{b'cover': b'https://cdn.cite.com.tw/images/a/A1010261.jpg', b'id': b'9789863840053', b'name': b'\xe6\xa5\xb5\xe9\x99\x90  \xe4\xb8\x8a'}


In [11]:
def crawl_book_isbn(isbn):
    # Initialize Chrome WebDriver
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 10)
    baseUrl = 'https://isbn.ncl.edu.tw/NEW_ISBNNet/H30_SearchBooks.php?&Pact=DisplayAll4Simple'
    
    try:
        driver.get(baseUrl)
        # driver.implicitly_wait(5)
        driver.maximize_window()
        
        
        navbar_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/nav/div/div[2]/ul[1]/li[3]/a')))
        navbar_button.click()

        # Wait for the dropdown to be visible and select ISBN
        select_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="FO_SearchField0"]')))
        select = Select(select_element)
        select.select_by_value('ISBN')

        # Enter the ISBN value
        search_input = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="FO_SearchValue0"]')))
        search_input.clear()
        search_input.send_keys(isbn)

        # Click the search button
        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/section/div/div/div[2]/div[2]/form[1]/div[2]/a')))
        search_button.click()
        
        xName = '//*[@id="ResultBox"]/table/tbody/tr[2]/td[3]/a'
        xCreator = '//*[@id="ResultBox"]/table/tbody/tr[2]/td[4]'
        xPublisher  = '//*[@id="ResultBox"]/table/tbody/tr[2]/td[5]'
        xDate = '//*[@id="ResultBox"]/table/tbody/tr[2]/td[6]'
        name =  wait.until(
                     EC.presence_of_element_located((By.XPATH, xName))
              ).text
        creator =  wait.until(
                     EC.presence_of_element_located((By.XPATH, xCreator))
              ).text
        publisher =  wait.until(
                     EC.presence_of_element_located((By.XPATH, xPublisher))
              ).text
        date =  wait.until(
                     EC.presence_of_element_located((By.XPATH, xDate))
              ).text
        driver.close()
        return name, creator, publisher, date
    except Exception as e:
        print(f"An error occurred: {e}")
        return 'error', 'error', 'error', 'error'
    

#


In [99]:
crawl_book_isbn('9789865562144')

('沙丘: 神帝= God emperor of dune',
 '法蘭克.赫伯特(Frank Herbert)著; 劉未央譯',
 '大家',
 '110/09')

In [46]:
start_checkpoint = 100
end_checkpoint = 150

In [3]:
isbnFile = open('isbns.txt', 'r')
isbns = isbnFile.readlines()
count = 0
isbnDataframe = pd.DataFrame(columns=["isbn","name","author","publisher", "date"])
for isbn_ in isbns:
    if count == end_checkpoint:
        break
    if count >= start_checkpoint:
        isbn = isbn_.strip()
        name, author, publisher, date = crawl_book_isbn(isbn)
        d = [isbn, name, author, publisher, date]
        isbnDataframe.loc[len(isbnDataframe)] = d
        print(d)
    count += 1
    
isbnDataframe.to_csv(f'./csv/isbn_{start_checkpoint}-{end_checkpoint}.csv')
    

NameError: name 'end_checkpoint' is not defined

In [38]:
def crawl_book_cover(isbn):
   service = Service()
   options = webdriver.ChromeOptions()
   # options = webdriver.SafariOptions()
   options.page_load_strategy = 'eager'
   options.add_argument("--enable-javascript")

   ua = UserAgent(platforms=['pc'], browsers=['chrome', 'edge', 'firefox', 'safari'])
   user_agent = ua.random
   options.add_argument(f'--user-agent={user_agent}')
   # options.add_argument('--headless')
   # Initialize Chrome WebDriver
   driver = webdriver.Chrome(service=service, options=options)
   # driver = webdriver.Safari(options=options)
   driver.maximize_window()
   wait = WebDriverWait(driver, 10)
   search_url = f"https://www.google.com/search?site=&tbm=isch&source=hp&biw=1873&bih=990&q={isbn}"
    
   try:
      driver.get(search_url)
      # driver.implicitly_wait(5)
      img_box = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="rso"]/div/div/div[1]/div/div/div[1]/div[2]/h3/a/div/div/div')))
      img_box.click()
      # XPath of the image display 
      fir_img = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Sva75c"]/div[2]/div[2]/div/div[2]/c-wiz/div/div[3]/div[1]/a/img[1]')))

      # fir_img = driver.find_element(By.XPATH, '//*[@id="Sva75c"]/div[2]/div[2]/div/div[2]/c-wiz/div/div[3]/div[1]/a/img[1]')
      # Retrieve attribute of src from the element
      img_src = fir_img.get_attribute('src')
      time.sleep(1)
      driver.close()
      
      
      return img_src
       
   except Exception as e:
      print(f"An error occurred: {e}")
      return 'error'
    

#


In [37]:
crawl_book_cover(9789577336033)

'https://s.eslite.com/upload/product/o/2611276536001/ec67093.jpg'

In [48]:
isbnFile = open('isbns.txt', 'r')
isbns = isbnFile.readlines()
count = 0
isbnDataframe = pd.DataFrame(columns=["isbn","cover"])
for isbn_ in isbns:
    if count == end_checkpoint:
        break
    if count >= start_checkpoint:
        isbn = isbn_.strip()
        cover = crawl_book_cover(isbn)
        d = [isbn, cover]
        isbnDataframe.loc[len(isbnDataframe)] = d
        print(d)
    count += 1
    
isbnDataframe.to_csv(f'./csv/cover_{start_checkpoint}-{end_checkpoint}.csv')

['9789570349689', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQscBPfazmL1TsyRPEqHBjJGIT_k4idle2hNA&s']
['9789570349856', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQNuLxOdvE5WzP9IfJOjjQpjDtpFIcZMv8N8Q&s']
['9789868112698', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRG7-4986-988wwc7lj2yUxVkMq0RbXKYC8Mw&s']
['9789866562198', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQAuyjouK_K6vzVtYQ49zCUsi4smN8EU5iRog&s']
['9789867600318', 'https://www.locuspublishing.com/webroot/file/books/1000/A010FM019-1000.png']
['9789577453495', 'https://im2.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/099/46/0010994692_bc_01.jpg&v=668bce71k&w=375&h=375']
['9789577453495', 'https://im2.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/099/46/0010994692_bc_01.jpg&v=668bce71k&w=375&h=375']
['9789862290873', 'https://s.eslite.com/Upload/Product/200906/o/633802254962602316.JPG']
['9789570494044', 'https://encrypted-tbn0.gstatic.com/images?q=tb